In [1]:
import pandas as pd
import snowflake.connector
import numpy as np


In [2]:
inv_ver = pd.read_csv('~/Desktop/.csv', dtype='str')


In [3]:
def add_check_digit(upc_str):
    odd_sum = 0
    even_sum = 0
    for i, char in enumerate(upc_str):
        j = i+1
        if j % 2 == 0:
            even_sum += int(char)
        else:
            odd_sum += int(char)

    total_sum = (odd_sum * 3) + even_sum
    mod = total_sum % 10
    check_digit = 10 - mod
    if check_digit == 10:
        check_digit = 0
    return upc_str + str(check_digit)
 
 


In [4]:
inv_ver['check_digit'] = inv_ver['UPC_VARIANT'].str[2:].str[:-1]
check_array = []
for i in inv_ver.itertuples():
    end = add_check_digit(i.check_digit)
    check_array.append(end)
inv_ver['VALID_UPC'] = check_array
inv_ver['INVALID_UPC'] = np.where(inv_ver['check_digit'] != inv_ver['VALID_UPC'],1,0)
inv_ver['GTIN'] = inv_ver['UPC_VARIANT']
 
inv_confirmed = inv_ver[inv_ver['INVALID_UPC'] != 0]

In [5]:
inv_confirmed

,UPC_VARIANT,GS1_COMPANY_PREFIX,PRODUCT_DESCRIPTION,MANUFACTURER_NAME,check_digit,VALID_UPC,INVALID_UPC,GTIN
0,00076420920051,0076420,PYRAMID OUTBURST IMPERIAL IPA 6PK/12OZ BOTTLES,FRATELLI,07642092005,076420920051,1,00076420920051
1,00076420921072,0076420,PYRAMID HEFEWEIZEN 6PK/12OZ BOTTLES,FRATELLI,07642092107,076420921072,1,00076420921072
2,00076420921096,0076420,PYRAMID APRICOT 6PK/12OZ BOTTLES,FRATELLI,07642092109,076420921096,1,00076420921096
3,00078250003379,0078250,STEEL RESERVE 211 HIGH GRAVITY LAGER BEER 4PK/...,MOLSON COORS BREWING COMPANY,07825000337,078250003379,1,00078250003379
4,00078250004482,0078250,STEEL RESERVE 6PK/4PK/16OZ CAN,MOLSON COORS BREWING COMPANY,07825000448,078250004482,1,00078250004482
5,00078250006110,0078250,STEEL RESERVE SPIKED PUNCH ALLOY SERIES 24 OZ CAN,MOLSON COORS BREWING COMPANY,07825000611,078250006110,1,00078250006110
6,00078250006356,0078250,STEEL RESERVE SPIKED TROPIC STORM ALLOY SERIES...,MOLSON COORS BREWING COMPANY,07825000635,078250006356,1,00078250006356
7,00079581184003,0079581,CALYPSO LEMONADE ORIGINAL 12PK/16OZ BOTTLE,KING JUICE CO.OF WIS.INC.,07958118400,079581184003,1,00079581184003
8,00079581184010,0079581,CALYPSO OCEAN BLUE LEMONADE 12PK/16OZ BOTTLES,KING JUICE CO.OF WIS.INC.,07958118401,079581184010,1,00079581184010
9,00079581184027,0079581,CALYPSO STRAWBERRY LEMONADE 12PK/16OZ BOTTLES,KING JUICE CO.OF WIS.INC.,07958118402,079581184027,1,00079581184027


In [ ]:
cnx = snowflake.connector.connect(user='*', password='*', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_XS', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')

 
con = cnx.cursor()
for i in inv_confirmed.itertuples():
    insert_query = f"""
    INSERT INTO DASCI.PRODUCT_MASTER_CATALOG (SOURCE_DESCRIPTION, SOURCE_PATH,  GTIN_12_UPC, PRODUCT_DESCRIPTION, UPDATE_USER, UPDATE_DATE)
        VALUES ('fdc_invoice','fdc_prod_live.fdc02.entity_invoice','{i.GTIN_12_UPC}',$${i.PRODUCT_DESCRIPTION}$$, 'rkothari', current_timestamp' )
    """
    try:
        con.execute(insert_query)
    except:
        print(i.GS1_COMPANY_PREFIX, i.VALID_UPC, i.PRODUCT_DESCRIPTION)
con.close()